In [1]:
import AdventOfCode
import Data.List.Split (splitOn)
import Data.List.Split (chunk)
import Data.List (foldl1')
import Data.Bits (xor)
import Data.Char (ord)
import Numeric (showHex)
import qualified Data.Vector as V

input <- map readInt . splitOn "," <$> dayString 10

circle = V.fromList [0..255]

In [2]:
subIndices vector index len 
    | (index + len) <= V.length vector = let
        indices = [index..(index+len-1)]
        in indices
    | (index + len) > V.length vector = let
        extra = (index+len) - V.length vector
        tillEnd = [index..(V.length vector - 1)]
        fromStart = [0..(extra-1)]
        indices = tillEnd ++ fromStart
        in indices

mutate vector index len = let
    indices   = V.fromList $ subIndices vector index len
    subvector = V.backpermute vector indices
    reversed  = V.reverse subvector
    updated   = V.update vector $ V.zip indices reversed
    in updated

loop orig 0 []     _     _          vector = vector
loop orig n []     index skipLength vector = loop orig (n-1) orig index skipLength vector
loop orig n (l:ls) index skipLength vector = let
    vector' = mutate vector index l
    index' = (index + l + skipLength) `mod` V.length vector
    skipLength' = skipLength + 1
    in loop orig n ls index' skipLength' vector'

In [3]:
finished = loop input 0 input 0 0 circle

product $ V.take 2 finished

5577

In [4]:
hash str = let
    suffixed = map ord str ++ [17, 31, 73, 47, 23]
    looped = loop suffixed 63 suffixed 0 0 circle
    chunked = chunk 16 $ V.toList looped
    hexed = map (flip showHex "" . foldl1' xor) chunked
    in concatMap (\h -> if length h == 1 then '0':h else h) hexed

In [5]:
hash <$> dayString 10

"44f4befb0f303c0bafd085f97741d51d"